# Introducción

El aprendizaje por refuerzo explora una aproximación *computacional* al aprendizaje por interacción. De la misma manera que cuando aprendemos a conducir estamos atentos a cómo reacciona nuestro entorno a nuestras acciones y buscamos maneras de influenciarlo a través de nuestro comportamiento, el aprendizaje por refuerzo estudia cómo agentes computacionales pueden desarrollar comportamientos inteligentes a través de este tipo de interacción.

## Aprendizaje por refuerzo

El aprendizaje por refuerzo, de ahora en adelante **axr**, consiste en aprender qué hacer (cómo relacionar situaciones y acciones) con el objetivo de maximizar una recompensa numérica. En ningún momento especificamos qué acciones debe tomar un *agente*, sino que le dejamos descubrir cuales son las que le darán una mayor recompensa. En la mayoría de situaciones, una acción no solo afectará a la recompensa inmediata, si no que tendrá un efecto en todas las situaciones futuras. Estas dos propiedades, búsqueda por prueba y error y futuras recompensas, son las más importantes del axr.

Para formalizar el problema del axr utilizamos ideas del campo de la teoría de sistemas dinámicos, en concreto el control óptimo de procesos de Markov incompletos. La idea básica consiste en aprender los aspectos más importantes sobre el problema real al que nuestro agente se enfrenta a través de la interacción con el entrono para conseguir su objetivo. El agente tiene que ser capaz de *precibir* su entorno y de llevar a cabo acciones que afecten a su estado. También necesita uno o varios objetivos relacionados con el estados del entorno. Un proceso de decisión de Markov incluyte estos tres aspectos: percepción, acción y objetivo. Cualquier método que sea capaz de resolver este tipo de problemas se considera como un método de axr.

El axr está considerado como un paradigma del machine learning diferente al aprendizaje supervisado y no supervisado. Se diferencia del aprendizaje supervisado en que no siempre será posible obtener ejemplos del comportamiento deseado para nuestro agente en cualquier tipo de situación en la que se pueda encontrar, por lo que deberá ser capaz de aprender de su propia experiencia. Por otro lado, se diferencia del aprendizaje no supervisado ya que éste no es capaz por si mismo de resolver el problema de maximización de la recompensa.

El principal problema al que nos enfrentamos en el axr es el balance entre **exploración** y **explotación**. Para obtener la máxima recompensa, un agente podría escoger aquellas acciones que ya conoce que y que le direon buenos resultados. Sin embargo, el hecho de explorar nuevas acciones podría, eventualmente, dar mucho mejor resultado. Así pues, nuestro agente tiene que ser capaz de explotar su conocimiento para obtener recompensa pero también de explorar para descubrir mejores acciones. El problema es que ninguna de las dos aproximaciones puede llevarse a cabo de manera independiente para resolver un problema. Un agente debe probar muchas acciones y, poco a poco, favorecer aquellas que parezcan ser mejores. Este problema sigue sin estar resuelto.

Otra propiedad importante que diferencia al axr de otro métodos es su capacidad de considerar todo el dominio del problema de un agente interactuando con su entrono, y no pequeñas partes o sub-tareas que puedan resolverse de manera independiente para llevar al objetivo final.

## Ejemplos

Algunos ejemplos interesantes de aplicaciones del axr son:

- Agentes que son capaces de jugar a juegos: ajedrez, go, atari, starcraft ...
- Sistemas de control adaptativo en entornos industriales: refinerías, fábricas, cadenas de montaje, ...
- Robótica
- Conducción autónoma: el coche recibe información de su entorno a través de sus cámaras y sensores y ejecuta comandos para acelerar, frenar, girar el volante, ...

Como vemos, todos los ejemplos comparten la existencia de un agente en constante interacción con su entorno para lograr un objetivo (a pesar de la posible incertidumbre). Tomar una decisión puede afectar a las acciones y oportunidades futuras, por lo que la elección de una accion requiere capacidad de planificación y predicción. Además, gracias a su interacción con el entorno, un agente puede adaptarse y aprender constantemente, ajustándose si es necesario para mejorar. De todas las formas de inteligencia computacional, el axr es el que más se asemeja a la forma en la que personas y animales actuamos.

## Elementos del axr

Además del *agente* y del *entorno*, existen cuatro subelementos esenciales en un sistema de axr:

- **política**: define el comportamiento de el agente en cada momento. Relaciona el estado que el agente percibe de su entorno con todas las posibles acciones que puede tomar. Puede ser tan simple como una función o una tabla, o tan complicada como un proceso de búsqueda. La política define completamente el comportamiento de un agente.
- **recompensa**: define el objetivo del problema, y es un un valor numérico que en cada momento el entorno envía al agente, el cual tiene el único objetivo de maximizarlo. Es el valor principal a tener en cuenta a la hora de alterar la política de un agente.
- **función de valor**: mientras que la recompensa indica la calidad de un estado de manera inmediata, la función de valor indica la calidad a largo plazo. El *valor* de un estado es la cantidad total de recompensa que un agente espera acumular en el futuro empezando en ese mismo estado. De esta manera, estados con una baja recompensa en relación a otros puede ser preferible si su valor es mayor (los estados futuros a los que nos permite llegar proporcionarán mayor recompensa). Así pues, favoreceremos acciones que impliquen mayor valor sobre recompensas inmediatas. El pricipal problema será estimar estos valores, ya que para ellos nuestro agente deberá explorar de manera repetida el entorno actualizando su información para poder llegar a ser capaz de tomar decisiones con fundamento.
- **modelo** del entorno: imita el comportamiento del entorno y sirve para planificar acciones considerando estados futuros que todavía no se han experimentado. El uso de un modelo del entorno nos permite dividir los métodos de axr en métodos sin modelos, en el que un agente aprende por prueba y error  (lo opuesto a la planificación) y métodos con modelo, aunque en varias aplicaciones esta línea es un poco difusa ya que se combinan ambas opciones.

## Ejemplo de aplicación: tres en raya

Con el objetivo de ilustrar la idea general del axr vamos a considerar un ejemplo en detalle: el juego del tres en raya. En este juego, dos jugadores se turnan para dibujar una X o una O en un tablero con 3x3 posiciones. El primer jugador en conseguir dibujar tres figuras en una línea horizontal, vertical u horizontal, gana. Nuestro objetivo es conseguir un agente que sea capaz de ganar siempre a este juego.

![](https://thumbs.gfycat.com/PoisedGrippingFox-small.gif)

In [30]:
import numpy as np

class Juego():
    def __init__(self, jugador1, jugador2, rondas=100):
        self.tabla = np.zeros((3,3))    # tabla vacía
        self.jugador1 = jugador1        # agente 1
        self.jugador2 = jugador2        # agente 2
        self.rondas = rondas
    
    def mover(self, simbolo, fila, columna):
        if self.tabla[fila, columna] == 0:
            self.tabla[fila, columna] = simbolo
        else:
            raise ValueError ("movimiento ilegal !")
    
    def posiciones_disponibles(self):
        return [(i, j) for j in range(3) for i in range(3) if self.tabla[i, j] == 0]
    
    def ganador(self):
        # comprobar filas y columnas
        if (self.tabla.sum(axis=0) == 3).sum() >= 1 or (self.tabla.sum(axis=1) == 3).sum() >= 1:
            return 1
        if (self.tabla.sum(axis=0) == -3).sum() >= 1 or (self.tabla.sum(axis=1) == -3).sum() >= 1:
            return -1 
        # comprobar diagonales
        diag_sums = [
            sum([self.tabla[i, i] for i in range(3)]),
            sum([self.tabla[i, 3 - i - 1] for i in range(3)]),
        ]
        if diag_sums[0] == 3 or diag_sums[1] == 3:
            return 1
        if diag_sums[0] == -3 or diag_sums[1] == -3:
            return -1        
        # empate
        if len(self.posiciones_disponibles()) == 0:
            return 0
        # seguir jugando
        return None
    
    def recompensa(self):
        # le damos 1 recompensa al jugador que gana
        ganador = self.ganador()
        if ganador == 1:
            self.jugador1.recompensa(1)
            self.jugador2.recompensa(0)
        elif ganador == -1:
            self.jugador1.recompensa(0)
            self.jugador2.recompensa(1)
        elif ganador == 0: # emapte
            self.jugador1.recompensa(0)
            self.jugador2.recompensa(0)
        else: # seguir jugando
            pass
        
    def jugar(self):
        for i in range(1,self.rondas+1):
            self.tabla = np.zeros((3,3))
            self.jugador1.posiciones = []
            self.jugador2.posiciones = []
            while True:
                # jugador 1
                accion = self.jugador1.mover(self.posiciones_disponibles(), self.tabla, 1)
                self.mover(1, accion[0], accion[1])
                self.jugador1.posiciones.append(str(self.tabla.reshape(3*3)))
                if self.ganador() is not None:
                    break
                # jugador 2
                accion = self.jugador2.mover(self.posiciones_disponibles(), self.tabla, 1)
                self.mover(-1, accion[0], accion[1])
                self.jugador2.posiciones.append(str(self.tabla.reshape(3*3)))
                if self.ganador() is not None:
                    break                    
            self.recompensa()
            
    def jugar2(self):
        self.tabla = np.zeros((3,3))
        while True:
            # jugador 1
            print(self.tabla)
            columna = int(input("Columna: "))
            fila = int(input("Fila: "))
            self.mover(-1, fila, columna)
            if self.ganador() is not None:
                break      
            # jugador 2
            accion = self.jugador1.mover(self.posiciones_disponibles(), self.tabla, 1, explore=False)
            self.mover(1, accion[0], accion[1])
            if self.ganador() is not None:
                break
        print(self.tabla)



Para resolver este juego con axr, en primer lugar definimos una tabla de números, uno por cada posible estado del juego. Cada numero respresentará la probabilidad de ganar el juego desde ese estado, el *valor* del estado. Así pues, la tabla sería la *función de valor*. Un estado $A$ es considerado mejor que un estado $B$ si el valor estimado de la probabilidad de ganar el juego desde $A$ es mayor que desde $B$. Si jugásemos con las Xs, todos los estados con tres X en raya tenría un valor de 1, ya que hemos ganado el juego. De la misma manera, cualquier estado con tres Os en raya tendría un valor de 0, hemos perdido. Para la inicialización de la tabla, podemos establecer el resto de valores en 0.5 (50% de posibilidades de ganar).

Nuestro agente jugará muchas partidas contra un oponente (que puede ser otro agente). En cada turno evaluamos los estados que resultarían de cada posible movimiento (posiciones no ocupadas) y elegimos aquella con un mayor *valor*. Ocasionalmente, elegiremos una acción aleatoria con el obnjetivo de explorar nuevos movimientos.

Mientras el agente va jugando, tendremos que actualizar la función de valor. Para ello, después de cada movimiento, cambiaremos el valor del estado del que venimos para que se acerque al valor del estado actual.

\begin{equation}
    V(S_t) \leftarrow V(S_t) + \alpha [V(S_{t+1}) - V(S_t)]
\end{equation}

donde $S_t$ denote el estado del que venimos, $S_{t+1}$ es el nuevo estado después del movimiento, $V(S_t)$ es el valor del estado $S_t$ y $\alpha$ es el ratio de aprendizaje.

In [31]:
class Agente():
    def __init__(self, alpha=0.2, prob_exp=0.3):
        self.funcion_de_valor = {} # tabla con pares estado -> valor
        self.alpha = alpha         # learning rate
        self.posiciones = []       # guardamos todas las posiciones de la partida
        self.prob_exp = prob_exp   # probabilidad de explorar
        
    def mover(self, posicions_disponibles, tabla, simbolo, explore=True):
        # exploracion
        if explore and np.random.uniform(0, 1) < self.prob_exp:
            # vamos a una posición aleatoria
            ix = np.random.choice(len(posicions_disponibles))
            return posicions_disponibles[ix]
        # explotacion
        # vamos a la posición con más valor
        valor_max = -1000
        for fila, columna in posicions_disponibles:
            siguiente_tabla = tabla.copy()
            siguiente_tabla[fila, columna] = simbolo
            estado = str(siguiente_tabla.reshape(3*3))
            valor = 0 if self.funcion_de_valor.get(estado) is None else self.funcion_de_valor.get(estado)
            if valor >= valor_max:
                valor_max = valor
                mejor_fila, mejor_columna = fila, columna
        return mejor_fila, mejor_columna
    
    def recompensa(self, recompensa):
        # al final de la partida (cuando recibimos la recompensa)
        # iteramos por tods los estados actualizando su valor en la tabla
        for p in reversed(self.posiciones):
            if self.funcion_de_valor.get(p) is None:
                self.funcion_de_valor[p] = 0
            self.funcion_de_valor[p] += self.alpha * (recompensa - self.funcion_de_valor[p])
            recompensa = self.funcion_de_valor[p]

In [32]:
agente1 = Agente()
agente2 = Agente()

juego = Juego(agente1, agente2, rondas=5000)

juego.jugar()

In [33]:
import pandas as pd

funcion_de_valor = sorted(agente1.funcion_de_valor.items(), key=lambda kv: kv[1], reverse=True)
tabla = pd.DataFrame({'estado': [x[0] for x in funcion_de_valor], 'valor': [x[1] for x in funcion_de_valor]})
tabla[:10]

,estado,valor
0,[ 0. 0. -1. 0. 0. -1. 1. 1. 1.],1.0
1,[ 1. 0. 0. 1. 0. -1. 1. 0. -1.],1.0
2,[ 1. 0. -1. 0. 1. -1. 0. 0. 1.],1.0
3,[ 1. 1. 1. 0. 0. -1. 0. 0. -1.],1.0
4,[ 0. -1. 1. 0. 1. -1. 1. -1. 1.],1.0
5,[ 0. 0. 0. 0. -1. -1. 1. 1. 1.],1.0
6,[ 1. 0. -1. 1. 0. -1. 1. -1. 1.],1.0
7,[-1. 0. 0. 0. 0. -1. 1. 1. 1.],1.0
8,[ 0. 1. 0. 0. 1. -1. 0. 1. -1.],1.0
9,[ 0. 0. 0. -1. 0. -1. 1. 1. 1.],1.0


Ahora podemos probar a jugar contra el agente a ver cómo se comporta.

In [34]:
juego.jugar2()

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Columna: 1
Fila: 1
[[ 0.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]]
Columna: 0
Fila: 1
[[ 0.  0.  0.]
 [-1. -1.  1.]
 [ 0.  0.  1.]]
Columna: 2
Fila: 0
[[ 0.  0. -1.]
 [-1. -1.  1.]
 [ 0.  1.  1.]]
Columna: 0
Fila: 2
[[ 0.  0. -1.]
 [-1. -1.  1.]
 [-1.  1.  1.]]


Este ejemplo sirve para ilustrar algunas de las propiedades clave del axr. En primer lugar, aprender a través de la interacción con el entorno (en este caso el otro agente). En segundo lugar, tenemos un objetivo claro y el comportamiento correcto del agente requiere de planificación y predicción que tenga en cuenta los efectos futuros de sus acciones.

## Resumen

El aprendizaje por refuerzo es una aproximación computacional a la comprensión y automatización del aprendizaje por objetivos y toma de decisiones. En esta aproximación, una agente aprende a través de la interacción directa con su entorno sin necesidad de supervisión explícita. Utiliza procesos de decisión de Markov para definir la interacción entre el agente y su entorno en términos del estado

## Historia

In [35]:
# TODO